<a href="https://colab.research.google.com/github/yagel2/TDS_NYC_Airbnb/blob/main/Final%20Project/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset (adjust the path or use a dataset URL)
df = pd.read_csv('your_dataset.csv')

# Preprocess: fill missing values, encode categorical columns
df = df.fillna(0)  # Example of filling missing values
df = pd.get_dummies(df)  # Convert categorical variables into dummy variables

# Split into features and target (change 'target_column' to your actual target column name)
X = df.drop(columns='target_column')
y = df['target_column']

# Show first few rows of the data
df.head()